In [4]:
import sys
sys.path.append('../scripts')

In [5]:
import os
# Disable weights and biases (if installed)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ["WANDB_DISABLED"] = "true"

In [6]:
from pathlib import Path

from dataset import load_data, get_dataloader
from generative.transformers_util import get_training_args, get_trainer, get_tokenizer

# Load Data & Seq2Seq Model

In [7]:
import hydra
from hydra import compose, initialize

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path=Path('..'), job_name='foo', version_base='1.1')
config = compose(config_name='experiment.yaml')

In [8]:
MODEL_PATH = "../data/ellipses/model/"

In [9]:
tokenizer = get_tokenizer(config)

/home/Florian.Borchert/miniconda3/envs/ellipses/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
base_path = Path('..')
train_df, val_df, test_df = load_data(base_path / config.data.cnf_tsv_path, base_path / config.data.controls_tsv_path)

# Error Analysis

In [11]:
from evaluation import error_analysis, Metrics
from transformers import Text2TextGenerationPipeline, AutoModelForSeq2SeqLM

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

In [13]:
pipeline = Text2TextGenerationPipeline(model, tokenizer=tokenizer, max_length=config.generation_max_length, device=0)

In [14]:
from evaluation import error_analysis, get_scores, encode_decode
from notebook_util import show_errors

In [15]:
import pandas as pd

def calculate_errors(out, sample):
    gen_text = [o['generated_text'] for o in out]
    errors = error_analysis(gen_text, encode_decode(sample.full_resolution, tokenizer), encode_decode(sample.raw_sentence, tokenizer))
    errors = pd.concat([errors, sample[['file', 'sentence_id']].reset_index()], axis=1)
    return errors

In [16]:
%%time
out_valid = pipeline(list(val_df.raw_sentence))

CPU times: user 9min 25s, sys: 305 ms, total: 9min 25s
Wall time: 9min 25s


In [17]:
errors_valid = calculate_errors(out_valid, val_df)
errors_valid.error_type.value_counts() / len(errors_valid)

tn         0.495420
tp         0.416320
replace    0.016653
delete     0.016653
fn         0.016653
insert     0.014988
fp         0.013322
complex    0.009992
Name: error_type, dtype: float64

In [23]:
%%time
scores = get_scores(errors_valid, "eval")
scores

CPU times: user 1min 21s, sys: 116 ms, total: 1min 22s
Wall time: 1min 24s


{'eval/exact_match': 0.9117402164862615,
 'eval/gleu': 0.9868297119100977,
 'eval/edit_distance_rel': 0.9539251318409139}

In [24]:
s = errors_valid[errors_valid.error_type == "insert"]
show_errors(s)

0;01_magenkarzinom_0092.tsv;3

__Input:__

Analysiert wurde das Ergebnis aller Patienten, bei denen zwischen 2010 und 2012 in Frankreich eine Ösophagus- oder Magenresektionen durchgeführt war (N=11.196).

__Error type:__ insert

1;02_kolorektales-karzinom_0100.tsv;4

...

__Error type:__ insert

17;29_adulte-weichgewebesarkome_0204.tsv;4

__Input:__

Der Stellenwert einer prä- oder postoperativen Strahlentherapie bei retroperitonealen Weichgewebesarkomen lässt sich aus den Ergebnissen einer randomiserten Studie sowie retrospektiven und prospektiven Fallsammlungen [REF], [REF], [REF], [REF] einschätzen.

__Error type:__ insert

In [25]:
%%time
out_test = pipeline(list(test_df.raw_sentence))

CPU times: user 9min 58s, sys: 412 ms, total: 9min 59s
Wall time: 9min 59s


In [26]:
errors_test = calculate_errors(out_test, test_df)
errors_test.error_type.value_counts()

tn         601
tp         448
delete      25
insert      25
fn          18
replace     16
fp          15
complex     11
Name: error_type, dtype: int64

In [27]:
%%time
scores = get_scores(errors_test, "test")
scores

CPU times: user 1min 21s, sys: 84 ms, total: 1min 21s
Wall time: 1min 24s


{'test/exact_match': 0.905090595340811,
 'test/gleu': 0.9778028196418292,
 'test/edit_distance_rel': 0.950224476907634}

# Some examples

In [39]:
def run_pipeline(examples):
    for e, p in zip(examples, pipeline(examples)):
        print(e, '-->', p['generated_text'])

In [40]:
# Forward ellipses
run_pipeline(['Krebs-Vorsorge / -Früherkennung', 'HPV31, 33, 45 und 51', 'Vitamin C, E und A1'])

Krebs-Vorsorge / -Früherkennung --> Krebs-Vorsorge / Krebs-Früherkennung
HPV31, 33, 45 und 51 --> HPV31, HPV33 und HPV51
Vitamin C, E und A1 --> Vitamin C, Vitamin E und Vitamin A1


In [41]:
# Backward ellipses
run_pipeline(['Chemo- und Strahlentherapie', 'BRAF- und MEK-Inhibitor', 'zielgerichtete und Immuntherapien'])

Chemo- und Strahlentherapie --> Chemotherapie und Strahlentherapie
BRAF- und MEK-Inhibitor --> BRAF-Inhibitor und MEK-Inhibitor
zielgerichtete und Immuntherapien --> zielgerichtete Therapien und Immuntherapien


In [43]:
# Complex ellipses
run_pipeline(['HPV-16- und/oder -18-Positivität', 'BRCA1/2-Mutation', 'Zweitlinien- oder Drittliniensystem- bzw. -chemotherapie'])

HPV-16- und/oder -18-Positivität --> HPV-16-Positivität und/oder HPV-18-Positivität
BRCA1/2-Mutation --> BRCA1-Mutation/BRCA2-Mutation
Zweitlinien- oder Drittliniensystem- bzw. -chemotherapie --> Zweitliniensystemtherapie oder Drittliniensystemtherapie bzw. Drittlinienchemotherapie


## Clean up

In [ ]:
%%time
out_train = pipeline(list(train_df.raw_sentence))

errors_train = calculate_errors(out_train, train_df)

errors_train.error_type.value_counts()

In [ ]:
errors_all = pd.concat([errors_train, errors_valid, errors_test])

In [ ]:
errors_all.error_type.value_counts()

In [ ]:
show_errors(errors_all[errors_all.error_type != "tp"].sort_values(['error_type', 'file', 'sentence_id']))